In [1]:
import pandas as pd
import sqlite3

import sys, os

import utils.mining_data_tb as md
import utils.sql_tb as sq

import warnings

warnings.filterwarnings("ignore")

# Database & Tables Creation

In [2]:
# Connection to database
connection = sqlite3.connect("health_canada.db")
# Extracting data from csv files
tables = md.data_to_tables("data/")
# Creating database and tables
sq.tables_to_sql(connection, tables)

# Database exploration

## JOIN

### 1. Foods in each food group 

Let's find out how many foods we have in each food group

In [3]:
query = """
SELECT fg.FoodGroupName as 'Food Group', COUNT(*) as 'Food Items'
FROM food_name fn
JOIN food_group fg
ON fn.FoodGroupId = fg.FoodGroupId
GROUP BY fg.FoodGroupId
ORDER BY COUNT(*) DESC;
"""

pd.read_sql_query(query, connection)

,Food Group,Food Items
0,Vegetables and Vegetable Products,785
1,Baked Products,441
2,Poultry Products,418
3,"Lamb, Veal and Game",363
4,Fruits and fruit juices,328
5,Finfish and Shellfish Products,325
6,Sweets,324
7,"Soups, Sauces and Gravies",267
8,Beverages,243
9,Dairy and Egg Products,241


### 2. Food items from each source

Let's see now what are the sources of this data and how many food items we are getting from those sources.

In [4]:
query = """
SELECT fs.FoodSourceDescription as 'Food Source', COUNT(*) as 'Food items'
FROM food_name as fn
JOIN food_source as fs
ON fn.FoodSourceId = fs.FoodSourceId
GROUP BY fn.FoodSourceId
ORDER BY COUNT(*) DESC;
"""

pd.read_sql_query(query, connection)

,Food Source,Food items
0,FOODS BASED ON DATA FROM USDA: NO CHANGES,3555
1,CNF Sampling and Analysis Program (SNAP-CAN),658
2,FOOD BASED ON DATA FROM USDA: SOME NUTRIENTS A...,356
3,"FOOD AVAILABLE IN THE CANADIAN FOOD SUPPLY, MA...",267
4,TRADITIONAL FOOD,212
5,FOOD BASED ON DATA FROM USDA: INFORMATION FROM...,139
6,"FOOD AVAILABLE IN THE CANADIAN FOOD SUPPLY, BU...",123
7,CNF RECIPE COMPILATION,112
8,FOOD BASED ON DATA FROM USDA: SOME NUTRIENTS C...,103
9,FOOD BASED ON DATA FROM USDA. FOOD HAS BEEN D...,70


## WHERE

### 3. Top food groups by nutrient

In this case, I want to see the food groups sorted by the amount of X nutrient.

In [5]:
# For this, I will first check out what nutrients we have and then choose a couple to find the top food groups for those nutrients
query = """
SELECT DISTINCT NutrientName
FROM nutrient_name
LIMIT 30;
"""

pd.read_sql_query(query, connection)

,NutrientName
0,PROTEIN
1,FAT (TOTAL LIPIDS)
2,"CARBOHYDRATE, TOTAL (BY DIFFERENCE)"
3,"ASH, TOTAL"
4,ENERGY (KILOCALORIES)
5,STARCH
6,SUCROSE
7,GLUCOSE
8,FRUCTOSE
9,LACTOSE


In order not to make this too long, let's try only 2: proteins and fats.

In [6]:
# Protein
query = """
SELECT fg.FoodGroupName as 'Food Group', AVG(na.NutrientValue) as 'Average Protein'
FROM food_group fg
JOIN food_name fn, nutrient_amount na, nutrient_name nn
ON fg.FoodGroupId = fn.FoodGroupId
AND fn.FoodId = na.FoodId
AND na.NutrientId = nn.NutrientId
WHERE nn.NutrientName = 'PROTEIN'
GROUP BY fg.FoodGroupId
ORDER BY AVG(na.NutrientValue) DESC;
"""

pd.read_sql_query(query, connection)

,Food Group,Average Protein
0,Beef Products,25.417824
1,Poultry Products,24.028780
2,"Lamb, Veal and Game",23.333636
3,Pork Products,22.663206
4,Finfish and Shellfish Products,21.264154
5,Sausages and Luncheon meats,16.306242
6,Legumes and Legume Products,15.670478
7,Nuts and Seeds,15.444892
8,Dairy and Egg Products,12.610830
9,Fast Foods,11.182126


In [7]:
# Protein
query = """
SELECT fg.FoodGroupName as 'Food Group', AVG(na.NutrientValue) as 'Average Fat'
FROM food_group fg
JOIN food_name fn, nutrient_amount na, nutrient_name nn
ON fg.FoodGroupId = fn.FoodGroupId
AND fn.FoodId = na.FoodId
AND na.NutrientId = nn.NutrientId
WHERE nn.NutrientName = 'FAT (TOTAL LIPIDS)'
GROUP BY fg.FoodGroupId
ORDER BY AVG(na.NutrientValue) DESC;
"""

pd.read_sql_query(query, connection)

,Food Group,Average Fat
0,Fats and Oils,67.187778
1,Nuts and Seeds,39.038058
2,Snacks,18.158710
3,Sausages and Luncheon meats,16.230671
4,Pork Products,15.057703
5,"Lamb, Veal and Game",13.650661
6,Dairy and Egg Products,13.349295
7,Fast Foods,12.445690
8,Baked Products,11.809751
9,Beef Products,11.506706


### 4. Foods in food group above a nutrient amount threshold

Let's find foods in a given food group that have an amount of a given nutrient above a given threshold.

We will just do one example of this.

In [8]:
# In this case, I want to find the less caloric nuts and seeds

query = """
SELECT fn.FoodDescription as 'Food', na.NutrientValue as 'Fat amount'
FROM food_group fg
JOIN food_name fn, nutrient_amount na, nutrient_name nn
ON fg.FoodGroupId = fn.FoodGroupId
AND fn.FoodId = na.FoodID
AND na.NutrientId = nn.NutrientId
WHERE fg.FoodGroupName = 'Nuts and Seeds'
AND nn.NutrientName = 'FAT (TOTAL LIPIDS)'
ORDER BY na.NutrientValue
LIMIT 30;
"""

pd.read_sql_query(query, connection)

,Food,Fat amount
0,"Nuts, Japanese chestnuts, boiled and steamed",0.19
1,"Nuts, coconut water (liquid from coconut)",0.20
2,"Nuts, Japanese chestnuts, raw",0.53
3,"Seeds, lotus seeds, raw",0.53
4,"Nuts, Chinese chestnuts, boiled and steamed",0.76
5,"Nuts, Japanese chestnuts, roasted",0.80
6,"Seeds, breadnuttree seeds, raw",0.99
7,"Nuts, Chinese chestnuts, raw",1.11
8,"Nuts, Chinese chestnuts, roasted",1.19
9,"Nuts, Japanese chestnuts, dried",1.24


## HAVING

### 5. Food groups with average nutrient above threshold

I want to find out which food groups have an average amount of a given nutrient above a given threshold.

In [9]:
# Let's see what food groups have more than 10g of protein in average

query = """
SELECT fg.FoodGroupName as 'Food group', AVG(na.NutrientValue) as 'Average nutrient amount'
FROM food_group fg
JOIN food_name fn, nutrient_amount na, nutrient_name nn
ON fg.FoodGroupId = fn.FoodGroupId
AND fn.FoodId = na.FoodID
AND na.NutrientId = nn.NutrientId
WHERE nn.NutrientName = 'PROTEIN'
GROUP BY fg.FoodGroupId
HAVING AVG(na.NutrientValue) > 10
ORDER BY AVG(na.NutrientValue) DESC;
"""

pd.read_sql_query(query, connection)

,Food group,Average nutrient amount
0,Beef Products,25.417824
1,Poultry Products,24.028780
2,"Lamb, Veal and Game",23.333636
3,Pork Products,22.663206
4,Finfish and Shellfish Products,21.264154
5,Sausages and Luncheon meats,16.306242
6,Legumes and Legume Products,15.670478
7,Nuts and Seeds,15.444892
8,Dairy and Egg Products,12.610830
9,Fast Foods,11.182126


In [10]:
# Let's do the same now, but with sugar and above 50mg
query = """
SELECT fg.FoodGroupName as 'Food group', AVG(na.NutrientValue) as 'Average nutrient amount'
FROM food_group fg
JOIN food_name fn, nutrient_amount na, nutrient_name nn
ON fg.FoodGroupId = fn.FoodGroupId
AND fn.FoodId = na.FoodID
AND na.NutrientId = nn.NutrientId
WHERE nn.NutrientName = 'CHOLESTEROL'
GROUP BY fg.FoodGroupId
HAVING AVG(na.NutrientValue) > 50
ORDER BY AVG(na.NutrientValue) DESC;
"""

pd.read_sql_query(query, connection)

,Food group,Average nutrient amount
0,"Lamb, Veal and Game",163.412162
1,Beef Products,140.394118
2,Pork Products,115.831731
3,Poultry Products,110.950372
4,Dairy and Egg Products,98.066390
5,Finfish and Shellfish Products,73.254417
6,Sausages and Luncheon meats,71.859060
7,Fast Foods,52.231214


### 6. Sources providing more than X amount of food to database

I want to find out which sources are providing more than a given amount of food items to the database.

In [11]:
# Food sources providing more than 100 food items
query = """
SELECT fs.FoodSourceDescription as 'Source', COUNT(*) as 'Food items'
FROM food_source fs
JOIN food_name fn
ON fs.FoodSourceId = fn.FoodSourceId
GROUP BY fs.FoodSourceId
HAVING COUNT(*) > 100
ORDER BY COUNT(*) DESC;
"""

pd.read_sql_query(query, connection)

,Source,Food items
0,FOODS BASED ON DATA FROM USDA: NO CHANGES,3555
1,CNF Sampling and Analysis Program (SNAP-CAN),658
2,FOOD BASED ON DATA FROM USDA: SOME NUTRIENTS A...,356
3,"FOOD AVAILABLE IN THE CANADIAN FOOD SUPPLY, MA...",267
4,TRADITIONAL FOOD,212
5,FOOD BASED ON DATA FROM USDA: INFORMATION FROM...,139
6,"FOOD AVAILABLE IN THE CANADIAN FOOD SUPPLY, BU...",123
7,CNF RECIPE COMPILATION,112
8,FOOD BASED ON DATA FROM USDA: SOME NUTRIENTS C...,103


## ARITHMETHIC OPERATORS

### 7. % of recommended daily intake

I want to know what % of the recommended daily intake the food groups fulfill for a given nutrient.

For the example, I'll use a 20 years old female. The recommended daily intake data source is the australian government:

[Female, 20 years old - recommended daily intake](https://www.eatforhealth.gov.au/node/1813927/done?sid=812350&token=57cde3dc19d19f3e4b59f8cf88757c13)

In [12]:
# Let's see what percentage of the recommended protein daily intake the different food groups fulfill
# For a 20 years old woman, this value is 46g/day
query = """
SELECT fg.FoodGroupName as 'Food group', (AVG(na.NutrientValue) / 46) * 100 as '% of the recommended daily intake'
FROM food_group fg
JOIN food_name fn, nutrient_amount na, nutrient_name nn
ON fg.FoodGroupId = fn.FoodGroupId
AND fn.FoodId = na.FoodId
AND na.NutrientId = nn.NutrientId
WHERE nn.NutrientName = 'PROTEIN'
GROUP BY fg.FoodGroupId
ORDER BY (AVG(na.NutrientValue) / 46) * 100 DESC;
"""

pd.read_sql_query(query, connection)

,Food group,% of the recommended daily intake
0,Beef Products,55.256138
1,Poultry Products,52.236478
2,"Lamb, Veal and Game",50.725296
3,Pork Products,49.267839
4,Finfish and Shellfish Products,46.226421
5,Sausages and Luncheon meats,35.448351
6,Legumes and Legume Products,34.066257
7,Nuts and Seeds,33.575852
8,Dairy and Egg Products,27.414848
9,Fast Foods,24.308971


In [13]:
# Now let's do the same but for Vitamin C
# For a 20 years old woman, this value is 45mg/day

query = """
SELECT fg.FoodGroupName as 'Food group', (AVG(na.NutrientValue) / 45) * 100 as '% of the recommended daily intake'
FROM food_group fg
JOIN food_name fn, nutrient_amount na, nutrient_name nn
ON fg.FoodGroupId = fn.FoodGroupId
AND fn.FoodId = na.FoodId
AND na.NutrientId = nn.NutrientId
WHERE nn.NutrientName = 'VITAMIN C'
GROUP BY fg.FoodGroupId
ORDER BY (AVG(na.NutrientValue) / 45) * 100 DESC;
"""

pd.read_sql_query(query, connection)

,Food group,% of the recommended daily intake
0,Fruits and fruit juices,71.029928
1,Spices and Herbs,69.495591
2,Vegetables and Vegetable Products,60.105398
3,Beverages,39.569387
4,Snacks,16.927536
5,Nuts and Seeds,12.071942
6,Babyfoods,9.449173
7,Mixed Dishes,8.629630
8,"Lamb, Veal and Game",6.384042
9,"Soups, Sauces and Gravies",5.283208


## NESTED QUERIES

### 8. Foods from specific food sources

I want to find out which foods come from which sources.

As before, I'm just going to do one example, as the rest would the same but changing names.

In [14]:
# I'll just pick any source:
# "FOOD BASED ON DATA FROM USDA: INFORMATION FROM USDA SURVEY FILES"
query = """
SELECT fn.FoodDescription as 'Food', fg.FoodGroupName as 'Food Group'
FROM food_name fn
JOIN food_group fg
ON fn.FoodGroupId = fg.FoodGroupId
WHERE FoodSourceId IN (
    SELECT FoodSourceId
    FROM food_source
    WHERE FoodSourceDescription = 'FOOD BASED ON DATA FROM USDA: INFORMATION FROM USDA SURVEY FILES'
);
"""

pd.read_sql_query(query, connection)

,Food,Food Group
0,"Alcohol, cocktail, scotch and soda",Beverages
1,"Sweets, syrups, grenadine",Sweets
2,"Drink, fruit flavour, powder, low calorie, vit...",Beverages
3,"Salad dressing, creamy, made with sour cream a...",Fats and Oils
4,"Cheese, cheddar, imitation, low cholesterol",Dairy and Egg Products
...,...,...
134,"Dessert, frozen, ice cream, vanilla, light, no...",Sweets
135,"Dessert, frozen yogourt, chocolate, skim milk,...",Sweets
136,"Dessert, frozen, juice, orange",Sweets
137,"Dessert, frozen, juice with cream",Sweets


### 9. Top foods by nutrient excluding food groups

I want to find out which foods are rich in a given nutrient excluding given food groups.

In [15]:
# I want to see the top protein foods
# Excluding Beef products

query = """
SELECT fn.FoodDescription as 'Food', fg.FoodGroupName as 'Food Group', na.NutrientValue as 'Nutrient Amount'
FROM food_group fg
JOIN food_name fn, nutrient_amount na, nutrient_name nn
ON fg.FoodGroupId = fn.FoodGroupId
AND fn.FoodId = na.FoodId
AND na.NutrientId = nn.NutrientId
WHERE nn.NutrientName = 'PROTEIN'
AND fg.FoodGroupId NOT IN (
    SELECT FoodGroupId
    FROM food_group
    WHERE FoodGroupName = 'Beef Products'
)
ORDER BY na.NutrientValue DESC
LIMIT 30;
"""

pd.read_sql_query(query, connection)

,Food,Food Group,Nutrient Amount
0,"Sweets, gelatin, dry powder, unsweetened",Sweets,85.60
1,"Game meat, native, bearded seal, (oogruk), air...","Lamb, Veal and Game",82.60
2,"Egg, chicken, white, dried, powder, glucose re...",Dairy and Egg Products,82.40
3,"Egg, chicken, white, dried",Dairy and Egg Products,81.10
4,Soy protein isolate (prepared with sodium),Legumes and Legume Products,80.69
5,Soy protein isolate (prepared with potassium),Legumes and Legume Products,80.69
6,"Game meat, native, moose, dried","Lamb, Veal and Game",79.50
7,"Beverage, protein powder, whey based, powder",Beverages,78.13
8,"Fish, steelhead trout, meat, dried",Finfish and Shellfish Products,77.27
9,"Game meat, native, narwhal, meat, dried","Lamb, Veal and Game",77.00


In [16]:
# Now let's do the same but exluding also Poultry Products

query = """
SELECT fn.FoodDescription as 'Food', fg.FoodGroupName as 'Food Group', na.NutrientValue as 'Nutrient Amount'
FROM food_group fg
JOIN food_name fn, nutrient_amount na, nutrient_name nn
ON fg.FoodGroupId = fn.FoodGroupId
AND fn.FoodId = na.FoodId
AND na.NutrientId = nn.NutrientId
WHERE nn.NutrientName = 'PROTEIN'
AND fg.FoodGroupId NOT IN (
    SELECT FoodGroupId
    FROM food_group
    WHERE FoodGroupName = 'Beef Products'
    OR FoodGroupName = 'Poultry Products'
)
ORDER BY na.NutrientValue DESC
LIMIT 30;
"""

pd.read_sql_query(query, connection)

,Food,Food Group,Nutrient Amount
0,"Sweets, gelatin, dry powder, unsweetened",Sweets,85.60
1,"Game meat, native, bearded seal, (oogruk), air...","Lamb, Veal and Game",82.60
2,"Egg, chicken, white, dried, powder, glucose re...",Dairy and Egg Products,82.40
3,"Egg, chicken, white, dried",Dairy and Egg Products,81.10
4,Soy protein isolate (prepared with sodium),Legumes and Legume Products,80.69
5,Soy protein isolate (prepared with potassium),Legumes and Legume Products,80.69
6,"Game meat, native, moose, dried","Lamb, Veal and Game",79.50
7,"Beverage, protein powder, whey based, powder",Beverages,78.13
8,"Fish, steelhead trout, meat, dried",Finfish and Shellfish Products,77.27
9,"Game meat, native, narwhal, meat, dried","Lamb, Veal and Game",77.00
